<a href="https://colab.research.google.com/github/Morsalah/M.Sc-Research-HRI-using-DIGIT-tactile-sensor/blob/main/Capturing_Images_Scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Capturing images from 2 digits

In [ ]:
import os
import time
import logging
from digit_interface.digit import Digit
from digit_interface.digit_handler import DigitHandler

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Function to create a directory for saving images if it doesn't exist
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")
    else:
        logger.info(f"Directory {path} already exists.")

# Function to capture and save frames from a DIGIT device
def capture_and_save_images_from_device(serial: str, save_path: str, num_images: int = 10, interval: float = 1.0):
    # Ensure the save directory exists
    create_directory(save_path)

    # Create the DIGIT object and connect to the device
    digit = Digit(serial)
    digit.connect()

    # Capture and save images
    for i in range(num_images):
        try:
            # Generate a timestamped filename for each image
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            image_filename = f"{save_path}/digit_{serial}_{timestamp}_{i+1}.png"

            # Save the current frame
            digit.save_frame(image_filename)
            logger.info(f"Image saved: {image_filename}")

            # Wait for the specified interval before capturing the next image
            time.sleep(interval)

        except Exception as e:
            logger.error(f"Error capturing image {i+1} from {serial}: {e}")

    # Disconnect the device after capturing images
    digit.disconnect()

# Main function to capture and save images from two devices
def capture_and_save_images_from_both_devices(serial_1: str, serial_2: str, save_path: str, num_images: int = 10, interval: float = 1.0):
    # Ensure the save directory exists
    create_directory(save_path)

    # Capture images from both devices
    capture_and_save_images_from_device(serial_1, save_path, num_images, interval)
    capture_and_save_images_from_device(serial_2, save_path, num_images, interval)

if __name__ == "__main__":
    # Specify the serial numbers of the two DIGIT devices
    serial_number_1 = "D21114"
    serial_number_2 = "D21115"

    # Specify the directory where images will be saved
    save_directory = "./captured_images_2digit"

    # Capture and save 10 images from both devices with a 1-second interval
    capture_and_save_images_from_both_devices(serial_number_1, serial_number_2, save_directory)


capturing images auotomaticlly

In [ ]:
import os
import time
import logging
import csv
import cv2
import numpy as np
from digit_interface.digit import Digit

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Function to create directories
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")
    else:
        logger.info(f"Directory {path} already exists.")

# Function to compare images and determine if touch is detected
def is_touch_detected(baseline_image, new_image, threshold=50):
    """
    Compare the new image with the baseline image.
    If the difference exceeds the threshold, classify it as 'touch detected'.
    """
    # Convert images to grayscale
    baseline_gray = cv2.cvtColor(baseline_image, cv2.COLOR_BGR2GRAY)
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Compute absolute difference
    diff = cv2.absdiff(baseline_gray, new_gray)

    # Calculate mean intensity of differences
    diff_score = np.mean(diff)

    logger.debug(f"Difference score: {diff_score}")

    # If the difference is above threshold, it's a touch (YES)
    return diff_score > threshold

# Capture and auto-label images
def capture_and_save_images(serial: str, save_path: str, num_images: int = 100, interval: float = 1.0):
    yes_path = os.path.join(save_path, "YES")
    no_path = os.path.join(save_path, "NO")
    create_directory(yes_path)
    create_directory(no_path)

    csv_filename = os.path.join(save_path, "labels.csv")
    csv_exists = os.path.exists(csv_filename)

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not csv_exists:
            writer.writerow(["filename", "label"])  # Add header if file is new

        # Connect to the DIGIT device
        digit = Digit(serial)
        digit.connect()

        # Capture baseline image (No Touch)
        print("Please ensure there is NO TOUCH for baseline image capture...")
        time.sleep(2)
        baseline_filename = os.path.join(save_path, "baseline.png")
        digit.save_frame(baseline_filename)
        baseline_image = cv2.imread(baseline_filename)
        logger.info(f"Baseline image saved: {baseline_filename}")

        yes_count, no_count = 0, 0

        for i in range(num_images):
            try:
                # Capture new frame
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_filename = f"{save_path}/digit_{serial}_{timestamp}_{i+1}.png"
                digit.save_frame(image_filename)

                # Read the newly captured image
                new_image = cv2.imread(image_filename)

                # Auto-label based on comparison
                if is_touch_detected(baseline_image, new_image):
                    label = "yes"
                    save_folder = yes_path
                    yes_count += 1
                else:
                    label = "no"
                    save_folder = no_path
                    no_count += 1

                # Move image to correct folder
                new_image_path = os.path.join(save_folder, os.path.basename(image_filename))
                os.rename(image_filename, new_image_path)

                # Log filename and label in CSV
                writer.writerow([new_image_path, label])
                logger.info(f"Image saved: {new_image_path} | Label: {label}")

                # Stop if 50 images per category are collected
                if yes_count >= num_images // 2 and no_count >= num_images // 2:
                    print("Collected required number of images for both categories.")
                    break

                # Wait for the specified interval
                time.sleep(interval)

            except Exception as e:
                logger.error(f"Error capturing image {i+1}: {e}")

        # Disconnect the device
        digit.disconnect()

if __name__ == "__main__":
    # Define parameters
    serial_number = "D21115"
    save_directory = "./captured_images"
    total_images = 100  # 50 YES and 50 NO images

    # Start capturing images with auto-labeling
    capture_and_save_images(serial_number, save_directory, num_images=total_images)


capturing images auotomaticlly - second

In [ ]:
import os
import time
import logging
import csv
import cv2
import numpy as np
from digit_interface.digit import Digit

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Function to create directories
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")
    else:
        logger.info(f"Directory {path} already exists.")

# Function to compare images and determine if touch is detected
def is_touch_detected(baseline_image, new_image, threshold=50):
    """
    Compare the new image with the baseline image.
    If the difference exceeds the threshold, classify it as 'touch detected'.
    """
    # Convert images to grayscale
    baseline_gray = cv2.cvtColor(baseline_image, cv2.COLOR_BGR2GRAY)
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Compute absolute difference
    diff = cv2.absdiff(baseline_gray, new_gray)

    # Calculate mean intensity of differences
    diff_score = np.mean(diff)

    logger.debug(f"Difference score: {diff_score}")

    # If the difference is above threshold, it's a touch (YES)
    return diff_score > threshold

# Capture and auto-label images
def capture_and_save_images(serial: str, save_path: str, num_images: int = 100, interval: float = 1.0):
    yes_path = os.path.join(save_path, "YES")
    no_path = os.path.join(save_path, "NO")
    create_directory(yes_path)
    create_directory(no_path)

    csv_filename = os.path.join(save_path, "labels.csv")
    csv_exists = os.path.exists(csv_filename)

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not csv_exists:
            writer.writerow(["filename", "label"])  # Add header if file is new

        # Connect to the DIGIT device
        digit = Digit(serial)
        digit.connect()

        # Capture baseline image (No Touch)
        print("Please ensure there is NO TOUCH for baseline image capture...")
        time.sleep(2)
        baseline_filename = os.path.join(save_path, "baseline.png")
        digit.save_frame(baseline_filename)
        baseline_image = cv2.imread(baseline_filename)
        logger.info(f"Baseline image saved: {baseline_filename}")

        yes_count, no_count = 0, 0

        for i in range(num_images):
            try:
                # Capture new frame
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_filename = f"{save_path}/digit_{serial}_{timestamp}_{i+1}.png"
                digit.save_frame(image_filename)

                # Read the newly captured image
                new_image = cv2.imread(image_filename)

                # Auto-label based on comparison
                if is_touch_detected(baseline_image, new_image):
                    label = "yes"
                    save_folder = yes_path
                    yes_count += 1
                else:
                    label = "no"
                    save_folder = no_path
                    no_count += 1

                # Move image to correct folder
                new_image_path = os.path.join(save_folder, os.path.basename(image_filename))
                os.rename(image_filename, new_image_path)

                # Log filename and label in CSV
                writer.writerow([new_image_path, label])
                logger.info(f"Image saved: {new_image_path} | Label: {label}")

                # Stop if 50 images per category are collected
                if yes_count >= num_images // 2 and no_count >= num_images // 2:
                    print("Collected required number of images for both categories.")
                    break

                # Wait for the specified interval
                time.sleep(interval)

            except Exception as e:
                logger.error(f"Error capturing image {i+1}: {e}")

        # Disconnect the device
        digit.disconnect()

if __name__ == "__main__":
    # Define parameters
    serial_number = "D21115"
    save_directory = "./captured_images"
    total_images = 100  # 50 YES and 50 NO images

    # Start capturing images with auto-labeling
    capture_and_save_images(serial_number, save_directory, num_images=total_images)


capturing images auotomaticlly - third

In [ ]:
import os
import time
import logging
import csv
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from digit_interface.digit import Digit

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Function to create directories
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")

# Function to compare images using SSIM + Pixel Difference
def is_touch_detected(baseline_image, new_image, ssim_threshold=0.8, pixel_change_threshold=5):
    """
    Uses both SSIM and pixel intensity difference to determine touch detection.
    """
    # Convert images to grayscale
    baseline_gray = cv2.cvtColor(baseline_image, cv2.COLOR_BGR2GRAY)
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Compute Structural Similarity Index (SSIM)
    similarity_score = ssim(baseline_gray, new_gray)

    # Compute absolute pixel difference
    diff = cv2.absdiff(baseline_gray, new_gray)
    changed_pixels = np.sum(diff > 20)  # Count pixels with significant change (threshold 20)
    total_pixels = diff.size
    pixel_change_percent = (changed_pixels / total_pixels) * 100

    logger.debug(f"SSIM score: {similarity_score}, Pixel change: {pixel_change_percent:.2f}%")

    # If SSIM is low OR pixel difference is high, classify as touch
    return similarity_score < ssim_threshold or pixel_change_percent > pixel_change_threshold

# Capture and auto-label images
def capture_and_save_images(serial: str, save_path: str, num_images: int = 100, interval: float = 1.0):
    yes_path = os.path.join(save_path, "YES")
    no_path = os.path.join(save_path, "NO")
    create_directory(yes_path)
    create_directory(no_path)

    csv_filename = os.path.join(save_path, "labels.csv")
    csv_exists = os.path.exists(csv_filename)

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not csv_exists:
            writer.writerow(["filename", "label"])  # Add header if file is new

        # Connect to the DIGIT device
        digit = Digit(serial)
        digit.connect()

        # Capture multiple baseline images (No Touch)
        print("Ensure NO TOUCH for baseline image capture...")
        time.sleep(2)
        baseline_images = []
        for _ in range(3):  # Capture 3 baseline images for better reference
            baseline_filename = os.path.join(save_path, f"baseline_{_}.png")
            digit.save_frame(baseline_filename)
            baseline_images.append(cv2.imread(baseline_filename))
            time.sleep(0.5)

        # Compute average baseline image
        baseline_image = np.mean(np.array(baseline_images), axis=0).astype(np.uint8)
        logger.info("Baseline images captured and averaged.")

        yes_count, no_count = 0, 0

        for i in range(num_images):
            try:
                # Capture new frame
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_filename = f"{save_path}/digit_{serial}_{timestamp}_{i+1}.png"
                digit.save_frame(image_filename)

                # Read the newly captured image
                new_image = cv2.imread(image_filename)

                # Auto-label based on SSIM + Pixel Difference
                if is_touch_detected(baseline_image, new_image):
                    label = "yes"
                    save_folder = yes_path
                    yes_count += 1
                else:
                    label = "no"
                    save_folder = no_path
                    no_count += 1

                # Move image to correct folder
                new_image_path = os.path.join(save_folder, os.path.basename(image_filename))
                os.rename(image_filename, new_image_path)

                # Log filename and label in CSV
                writer.writerow([new_image_path, label])
                logger.info(f"Image saved: {new_image_path} | Label: {label}")

                # Stop if 50 images per category are collected
                if yes_count >= num_images // 2 and no_count >= num_images // 2:
                    print("Collected required number of images for both categories.")
                    break

                # Wait for the specified interval
                time.sleep(interval)

            except Exception as e:
                logger.error(f"Error capturing image {i+1}: {e}")

        # Disconnect the device
        digit.disconnect()

if __name__ == "__main__":
    # Define parameters
    serial_number = "D21114"
    #serial_number = "D21115"
    save_directory = "./captured_images"
    total_images = 10  # 50 YES and 50 NO images

    # Start capturing images with auto-labeling
    capture_and_save_images(serial_number, save_directory, num_images=total_images)


capturing images yes or no folders

In [ ]:
import os
import time
import logging
import csv
import cv2
import numpy as np
from digit_interface.digit import Digit

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Function to create directories
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")

# Capture and save images with user-specified label
def capture_images(serial: str, save_path: str, num_images: int = 50, interval: float = 1.0):
    yes_path = os.path.join(save_path, "YES")
    no_path = os.path.join(save_path, "NO")
    create_directory(yes_path)
    create_directory(no_path)

    # Ask user whether capturing for YES or NO
    label = input("Enter 'YES' if capturing touch images, 'NO' for no-touch images: ").strip().upper()
    while label not in ["YES", "NO"]:
        print("Invalid input. Please enter 'YES' or 'NO'.")
        label = input("Enter 'YES' if capturing touch images, 'NO' for no-touch images: ").strip().upper()

    save_folder = yes_path if label == "YES" else no_path

    csv_filename = os.path.join(save_path, "labels.csv")
    csv_exists = os.path.exists(csv_filename)

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not csv_exists:
            writer.writerow(["filename", "label"])  # Add header if file is new

        # Connect to the DIGIT device
        digit = Digit(serial)
        digit.connect()
        print(f"Capturing {num_images} images for {label} category...")

        for i in range(num_images):
            try:
                # Capture new frame
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_filename = f"digit_{serial}_{timestamp}_{i+1}.png"
                full_image_path = os.path.join(save_folder, image_filename)
                digit.save_frame(full_image_path)

                # Log filename and label in CSV
                writer.writerow([full_image_path, label.lower()])
                logger.info(f"Image saved: {full_image_path} | Label: {label.lower()}")

                # Wait for the specified interval
                time.slee
                p(interval)

            except Exception as e:
                logger.error(f"Error capturing image {i+1}: {e}")

        # Disconnect the device
        digit.disconnect()
        print(f"Finished capturing {num_images} images for {label} category.")

if __name__ == "__main__":
    # Define parameters
    serial_number = "D21115"
    save_directory = "./captured_images"
    total_images = 400  # Adjust as needed

    # Start capturing images
    capture_images(serial_number, save_directory, num_images=total_images)


capture image

In [ ]:
import os
import time
import logging
import csv
from digit_interface.digit import Digit

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Create directories if they don't exist
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)
        logger.info(f"Directory {path} created.")
    else:
        logger.info(f"Directory {path} already exists.")

# Capture and save images with labels
def capture_and_save_images(serial: str, save_path: str, num_images: int = 100, interval: float = 1.0):
    yes_path = os.path.join(save_path, "YES")
    no_path = os.path.join(save_path, "NO")
    create_directory(yes_path)
    create_directory(no_path)

    # Create or open CSV file for labeling
    csv_filename = os.path.join(save_path, "labels.csv")
    csv_exists = os.path.exists(csv_filename)

    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not csv_exists:
            writer.writerow(["filename", "label"])  # Add header if file is new

        # Connect to the DIGIT device
        digit = Digit(serial)
        digit.connect()

        yes_count, no_count = 0, 0

        for i in range(num_images):
            try:
                # Ask user for label (touch or no touch)
                label = input("Is this a 'YES' (touch) or 'NO' (no touch) image? Type 'yes' or 'no': ").strip().lower()
                while label not in ["y", "n"]:
                    print("Invalid input! Please type 'yes' or 'no'.")
                    label = input("Is this a 'YES' (touch) or 'NO' (no touch) image? ").strip().lower()

                if label == "y":
                    save_folder = yes_path
                    yes_count += 1
                else:
                    save_folder = no_path
                    no_count += 1

                # Generate a timestamped filename
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                image_filename = f"{save_folder}/digit_{serial}_{timestamp}_{i+1}.png"

                # Capture and save image
                digit.save_frame(image_filename)
                logger.info(f"Image saved: {image_filename}")

                # Log filename and label in CSV
                writer.writerow([image_filename, label])

                # Stop if we've collected 50 images per category
                if yes_count >= num_images // 2 and no_count >= num_images // 2:
                    print("Collected required number of images for both categories.")
                    break

                # Wait for the specified interval
                time.sleep(interval)

            except Exception as e:
                logger.error(f"Error capturing image {i+1}: {e}")

        # Disconnect the device
        digit.disconnect()

if __name__ == "__main__":
    # Define parameters
    serial_number = "D21115"
    save_directory = "./captured_images"
    total_images = 100  # 50 YES and 50 NO images

    # Start capturing images
    capture_and_save_images(serial_number, save_directory, num_images=total_images)
